<a href="https://colab.research.google.com/github/Fahim-Anin/Machine_Learning_Internship/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

In [2]:
import torch
import torch.nn as nn
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:

a= "How are you?"
a=tokenize(a)
b=[stem(i) for i in a]
print(b)

['how', 'are', 'you', '?']


In [4]:
import requests
from pathlib import Path
import json
# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("intents.json").is_file():
  print("intents.json already exists, skipping download")
else:
  print("Downloading intents.json")
  request = requests.get("https://raw.githubusercontent.com/patrickloeber/pytorch-chatbot/master/intents.json")
  with open("intents.json", "wb") as f:
    f.write(request.content)


In [5]:
with open("intents.json", "r") as f:
    intents_data = json.load(f)

tags = []
patterns = []

for intent in intents_data["intents"]:
    tags.append(intent["tag"])
    patterns.extend(intent["patterns"])

# Now, 'tags' contains the tags, and 'patterns' contains all patterns
print("Tags:", tags)
print("Patterns:", patterns)

Tags: ['greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
Patterns: ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", "Thank's a lot!", 'Which items do you have?', 'What kinds of items are there?', 'What do you sell?', 'Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?', 'How long does delivery take?', 'How long does shipping take?', 'When do I get my delivery?', 'Tell me a joke!', 'Tell me something funny!', 'Do you know a joke?']


In [6]:
from torch.utils.data import Dataset, DataLoader

all_words = []
tags = []
xy = [] #for differentiate pattern wth specific tags
# loop through each sentence in our intents patterns
for intent in intents_data['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]

# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print("Tags:",tags)
print("lowes+stem:", all_words)
print("Patterns+tags:", xy)


Tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
lowes+stem: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
Patterns+tags: [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'), (['Which', 'items', 'do', 'you', 'have', '?'], 'items